In [36]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import ElectraForSequenceClassification, ElectraConfig, AutoTokenizer, ElectraModel

from konlpy.tag import Mecab

In [37]:
MODEL_NAME = "monologg/koelectra-small-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
electra_config = ElectraConfig.from_pretrained(MODEL_NAME)
model = ElectraModel.from_pretrained(MODEL_NAME)

In [38]:
data_df = pd.read_csv("../data/train/train.tsv", sep="\t", header=None)
data_df

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
8995,wikipedia-5414-12-15-21-0-4,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,15,21,2002년,0,4,관계_없음
8996,wikipedia-10384-4-12-14-0-1,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,12,14,일본,0,1,단체:본사_국가
8997,wikipedia-25913-6-8-10-93-106,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,8,10,선무원종공신(宣武原從功臣),93,106,인물:직업/직함
8998,wikitree-12062-15-0-3-46-47,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,0,3,북미,46,47,관계_없음


### Tokenize 된 Sequence의 최대 길이(토큰의 개수)

In [39]:
max_len = 0
for sent in data_df.iloc[:, 1]:
    encoded_sent = tokenizer.encode(sent)
    max_len = max(max_len, len(encoded_sent))
print(max_len)

267


# Tokenize 된 Sequence에서 Entity에 해당하는 토큰 index 찾기

In [40]:
class attrib:
    def __init__(self, data_df, data_idx):
        self.sentence = data_df.iloc[data_idx, 1]

        self.e1_name = data_df.iloc[data_idx, 2]
        self.e1_si = data_df.iloc[data_idx, 3]
        self.e1_ei = data_df.iloc[data_idx, 4]

        self.e2_name = data_df.iloc[data_idx, 5]
        self.e2_si = data_df.iloc[data_idx, 6]
        self.e2_ei = data_df.iloc[data_idx, 7]

        self.relation = data_df.iloc[data_idx, 8]
        
        self.cvt_idx()
        
    def cvt_idx(self):
        self.e1_si -= self.sentence[:self.e1_si].count(" ")
        self.e1_ei -= self.sentence[:self.e1_ei].count(" ")

        self.e2_si -= self.sentence[:self.e2_si].count(" ")
        self.e2_ei -= self.sentence[:self.e2_ei].count(" ")

In [71]:
for i in range(len(data_df)):
# for i in range(56, 57):
    info = attrib(data_df, i)
    
    origin_sent = info.sentence
    tokenized_sent = tokenizer.tokenize(info.sentence)
    encoded_sent = tokenizer.encode(info.sentence)
    
#     print(f"entity 1 name : {info.e1_name}")
#     print(f"entity 1 start idx : {info.e1_si}")
#     print(f"entity 1 end idx : {info.e1_ei}")
#     print(f"entity 2 name : {info.e2_name}")
#     print(f"entity 2 start idx : {info.e2_si}")
#     print(f"entity 2 end idx : {info.e2_ei}")
    
#     print(f"original Sentence : {origin_sent}")
#     print(f"tokenized Sentence : {tokenized_sent}")
#     print(f"encoded Sentence : {encoded_sent}")
    
    e1_idx_ = []
    e2_idx_ = []

    count = 0
    for idx, word in enumerate(tokenized_sent):
        for word_i in range(len(word) - word.count("#")):
            if count + word_i in range(info.e1_si, info.e1_ei + 1):
                e1_idx_.append(idx)
                break
        for word_i in range(len(word) - word.count("#")):
            if count + word_i in range(info.e2_si, info.e2_ei + 1):
                e2_idx_.append(idx)
                break
#         if count in range(info.e1_si, info.e1_ei + 1):
#             e1_token_idx.append(idx)
#         if count in range(info.e2_si, info.e2_ei + 1):
#             e2_token_idx.append(idx)

        count += len(word)
        count -= word.count("#")
    
    e1_is_ = [True if i in e1_idx_ else False for i in range(288)]
    e2_is_ = [True if i in e2_idx_ else False for i in range(288)]

#     print(f"entity 1 token index : {e1_token_idx}")
#     print(f"entity 2 token index : {e2_token_idx}")
    
#     print(f"entity 1 tokens : {[tokenized_sent[i] for i in e1_idx_]}")
#     print(f"entity 2 tokens : {[tokenized_sent[i] for i in e2_idx_]}")
    
#     print("\n" + "=" * 200 + "\n")
    
    if not any(e1_is_) or not any(e2_is_):
        raise Exception(f"{i} / {e1_idx_} / {e2_idx_}")

In [58]:
tokenized_sentence = tokenizer(
            text=data_df.iloc[:, 1].to_list(),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=312,
            add_special_tokens=True,
        )
print(tokenized_sentence.input_ids.shape)

torch.Size([9000, 267])
